<a href="https://colab.research.google.com/github/kamalkant9928/BOOK_RECOMMEND_SYSTEM/blob/main/Book_Recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df1=pd.read_csv('/content/Books.csv')
df1

In [ ]:
df1.isnull().sum()

In [ ]:
df1.duplicated().sum()

In [ ]:
df2=pd.read_csv('Users.csv')
df2

In [ ]:
df2.isnull().sum()

In [ ]:
df2.duplicated().sum()

In [ ]:
df3=pd.read_csv('Ratings.csv')
df3

In [ ]:
df3.isnull().sum()

In [ ]:
df3.duplicated().sum()

In [ ]:
rat_book=df3.merge(df1,on='ISBN')
rat_book

In [ ]:
num_rating_df = rat_book.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'}, inplace=True)
num_rating_df

In [ ]:
avg_rating_df = rat_book.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_ratings'}, inplace=True)
avg_rating_df

In [ ]:
popular_df=avg_rating_df.merge(num_rating_df,on='Book-Title')

In [ ]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings', ascending=False).head(50)
popular_df

In [ ]:
popular_df = popular_df.merge(df1, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_ratings']]


In [ ]:
x = rat_book.groupby('User-ID').count()['Book-Rating'] > 200
print(x[x])
rated_users = x[x].index
rated_users

In [ ]:
filtered_rating = rat_book[rat_book['User-ID'].isin(rated_users)]
filtered_rating

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
print(y[y])
famous_books = y[y].index
famous_books

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
final_ratings

In [ ]:
pt = final_ratings.pivot_table(index='Book-Title', columns ='User-ID', values='Book-Rating')
pt

In [ ]:
pt.fillna(0, inplace=True)
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)

In [ ]:
similarity_scores[0]

In [ ]:
similarity_scores.shape

In [ ]:
def recommend(book_name):
    if book_name not in pt.index:
        return f"Book '{book_name}' not found in the recommendation database. Please check the spelling or try another book."

    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = df1[df1['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    return data

In [ ]:
recommend('Message in a Bottle')

In [ ]:
recommend("River's End")